from https://arxiv.org/pdf/2106.11202.pdf, Table 6

In [1]:
import pysm3.units as u

In [2]:
fwhm = [1.57, 1.17, 1.04] * u.arcmin

In [3]:
freqs = [93.8, 147.0, 219.9] * u.GHz

In [4]:
bandwidth = [26.4, 32.5, 53.6] * u.GHz

In [5]:
import numpy as np

In [6]:
from collections import OrderedDict
from astropy import units as u
from astropy.table import QTable

In [7]:
ipac_warning = [
    "Text file in IPAC table format, read with astropy",
    "from astropy.table import QTable",
    "QTable.read('filename.tbl', format='ascii.ipac')",
    f"Instrument model exported from `Arxiv 2106.11202`",
]

In [8]:
band = ["95GHz", "150GHz", "220GHz"]

In [9]:
table = QTable(
    data=[band, freqs, fwhm, bandwidth],
    names=["band", "center_frequency", "fwhm", "bandwidth"],
)

In [10]:
table["nside"] = 0
table

band,center_frequency,fwhm,bandwidth,nside
,GHz,arcmin,GHz,
str6,float64,float64,float64,int64
95GHz,93.8,1.57,26.4,0
150GHz,147.0,1.17,32.5,0
220GHz,219.9,1.04,53.6,0


In [11]:
table.add_index("band")

In [12]:
cd ../../private/202308_panexp_spt3g

/mnt/home/azonca/p/software/s4mapbasedsims/private/202308_panexp_spt3g


In [13]:
table.write("spt_instrument_model.ecsv", overwrite=True)

In [14]:
table.meta["comments"] = ipac_warning

In [15]:
table.write(
    "spt_instrument_model.tbl",
    format="ascii.ipac",
    overwrite=True,
)